# Demand side flexibility


In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [3]:
SCALE_MWH = 1000
SCALE_DOLLAR = 1000;

In [4]:
fixed_cost = CSV.read("fixed_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Fixed Cost,Fixed OM cost
,Float64,Float64
1,543.5,21.7531
2,666.821,28.7795
3,1110.5,37.5
4,3415.0,103.5
5,434.537,0.0
6,186.085,23.3723


In [5]:
variable_cost = CSV.read("variable_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Variable OM,Ramp,Start
,Float64,Float64,Float64
1,0.0,0.0,0.0
2,0.0,0.0,0.002
3,0.00589,0.006,0.035
4,0.011176,0.015,0.07
5,0.0,0.001,0.0
6,0.0,0.0,0.0


In [6]:
co2_intensity = CSV.read("co2_intensity.csv",DataFrame)

Row,CO2 Intensity
,Int64
1,0
2,0
3,452
4,965
5,0
6,0


In [7]:
P_max_s = 95897829.43 / SCALE_MWH
P_max_w = 11158008 / SCALE_MWH;
P_max_s, P_max_w

(95897.82943000001, 11158.008)

In [8]:
Availability_matrix = CSV.read("Availability.csv", DataFrame);
println(size(Availability_matrix))

(8760, 5)


In [9]:
demand = CSV.read("demand.csv", DataFrame) ./ SCALE_MWH;
println(size(demand))
first(demand, 5)

(8760, 4)


Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,177.72,109.564,169.055,55.3949
2,174.776,108.33,164.425,37.6361
3,173.069,102.035,165.18,33.1084
4,171.107,96.909,168.945,29.3248
5,171.561,94.8884,173.68,23.6573


In [10]:
ramp_limits = CSV.read("ramp_limits.csv", DataFrame)

Row,Ramp_limit
,Float64
1,1.0
2,0.1
3,0.025
4,0.03
5,1.0
6,1.0


In [11]:
curtailment_costs = [0, 2, 0, 0, 0, 0] ./ SCALE_DOLLAR

6-element Vector{Float64}:
 0.0
 0.002
 0.0
 0.0
 0.0
 0.0

In [12]:
hrs = 4000
hre = 5000
sel_hrs = hrs:hre
println(sel_hrs)
techs = 4
n_eu = 4
# pad year by 1 day
add_h = 24

#Change to consider only RE + Battery
C_FC = fixed_cost[[1,2,5],1]
C_FOM = fixed_cost[[1,2,5],2]
E_CO2 = co2_intensity[[1,2,5],1]
C_VOM = variable_cost[[1,2,5],1]
C_ramp = variable_cost[[1,2,5],2]
C_start = variable_cost[[1,2,5],3]
C_curt = curtailment_costs[[1,2,5]]

D_b = demand[sel_hrs, 1:4] # Baseline demand with no flex
println(size(D_b))
D_b = vcat(reverse(reverse(demand)[1:add_h, 1:n_eu]), D_b, D_b[1:24, 1:n_eu])
println(size(D_b))
total_demand = sum(eachcol(D_b))
X_D = total_demand

A_r = ramp_limits[1:techs, 1]

A_s = Availability_matrix[sel_hrs,1:techs]
A_s = vcat(reverse(reverse(Availability_matrix)[1:add_h, 1:n_eu]), A_s, A_s[1:24, 1:n_eu])
println(size(A_s))
# over time, opt each year with worst day for availability and highest demand?

4000:5000
(1001, 4)
(1049, 4)
(1049, 4)


# Constant

In [13]:
eta = 0.8; # Efficiency of the battery
CO2_price = 100; # price of the CO2 per ton
H = 4;

# Creating the model

List of variable 

In [14]:
Power_generation_ls = 1:3 # Solar, Wind, Battery

fossil_fuel_ls = 1:2 # CCGT, Coal

nb_hours_ls = 1:size(X_D,1)

H = 20

20

In [15]:
model = Model(Gurobi.Optimizer)
println("Declare Model")
set_optimizer_attribute(model, "NonConvex", 2)
#model = Model(NLopt.Optimizer)
#set_optimizer_attribute(model, "algorithm", :LD_MMA)

#The installed power of the mix
@variable(model, P[Power_generation_ls] >= 0)


# Installed capacity renewables < max capacity
println("Renewable maximum installed capacity constraint")
@constraint(model, Con_sgen, P[1] <= P_max_s*10);
@constraint(model, Con_wgen, P[2] <= P_max_w*10);

@constraint(model, Power_battery_lb, P[3]>= 1000)
#@constraint(model, Power_battery_ub, P[3]<= 10000)
#Generation of solar, wind, coal, CCGT
@variable(model, X_gen[nb_hours_ls, Power_generation_ls ] >= 0)


@variable(model, X_soc[nb_hours_ls]>=0)
@variable(model, X_ch[nb_hours_ls]>=0)

println("Objective function")
CO2_price = 100
# The cost of the mix assuming a certain carbon cost
#@objective(model, Min, sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)) +
#sum(sum((C_VOM[j] + CO2_price * E_CO2[j])* X_gen[t,j] for j in 1:size(P,1)) for t in 1:size(X_gen,1)))


@objective(model, Min, 
    (
        sum((C_FC[i] + C_FOM[i])*P[i] for i in Power_generation_ls) +
        sum(sum((C_VOM[i] + CO2_price * E_CO2[i])* X_gen[t, i] for i in Power_generation_ls) for t in nb_hours_ls)
));

# For each tech, at each hour, usage cannot exceed available capacity
println("Availability & installed power constraint")
@constraint(model, Max_production[i in Power_generation_ls, t in nb_hours_ls], X_gen[t, i] <= P[i]*A_s[t, i])

# Supply meets demand at each hours:
println("Supply meets demand constraint")
@constraint(model, Supply_demand[t in nb_hours_ls], sum(X_gen[t,i] for i in Power_generation_ls) >= X_D[t]);


Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Declare Model
Set parameter NonConvex to value 2
Renewable maximum installed capacity constraint
Objective function
Availability & installed power constraint
Supply meets demand constraint


![Alt text](image.png)

Variable to indicate if $x_{g,t}<x_{d,t}$

In [16]:
M = 10000000000000


@variable(model, Z_X_gen_lower_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_lower_X_D_1[t in nb_hours_ls], X_D[t] - sum(X_gen[t,i] for i in 1:2)  <= M*Z_X_gen_lower_X_D[t])
@constraint(model, Z_X_gen_lower_X_D_2[t in nb_hours_ls], -X_D[t] + sum(X_gen[t,i] for i in 1:2) <= M*(1-Z_X_gen_lower_X_D[t]));


Create the variable X_min = min(X_1, X_2, X_3)

In [17]:
# X_1, X_2, X_3 = [eta*(sum(X_gen[t,i] for i in Power_generation_ls)- X_D[t]) , eta*P3], (1- X_soc[t])*P[5]* H]
@variable(model, z1[t in nb_hours_ls], Bin)
@variable(model, z2[t in nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Z_1 expression")
@constraint(model, z_1_1[t in nb_hours_ls], eta*P[3] - eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*z1[t])
@constraint(model, z_1_2[t in nb_hours_ls], -eta*P[3] + eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-z1[t]))


#Minimum between X_1 and X_2
@variable(model, min_X_1_X_2[nb_hours_ls])

println("Min X_1, X_2 expression")
@constraint(model, X_1_X_2_lower_1[t in nb_hours_ls], min_X_1_X_2[t] <= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]))
@constraint(model, X_1_X_2_lower_2[t in nb_hours_ls], min_X_1_X_2[t] <= eta*P[3])
@constraint(model, X_1_X_2_higher_1[t in nb_hours_ls], min_X_1_X_2[t] >= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-z1[t]))
@constraint(model, X_1_X_2_higher_2[t in nb_hours_ls], min_X_1_X_2[t] >= eta*P[3] - M*z1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, z_2_1[t in nb_hours_ls], (1- X_soc[t])* P[3] * H - min_X_1_X_2[t] <=  M*z2[t])
@constraint(model, z_2_2[t in nb_hours_ls], - (1- X_soc[t]) * P[3] * H + min_X_1_X_2[t] <= M*(1-z2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_X_1_X_2_X_3[nb_hours_ls])

@constraint(model, X_1_X_2_X_3_lower_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= min_X_1_X_2[t])
@constraint(model, X_1_X_2_X_3_lower_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= (1- X_soc[t])*P[3]* H)
@constraint(model, X_1_X_2_X_3_higher_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= min_X_1_X_2[t] - M*(1-z2[t]))
@constraint(model, X_1_X_2_X_3_higher_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= (1- X_soc[t])*P[3]* H - M*z2[t]);



Z_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_ch

In [18]:
# Need to put this conditions above
@variable(model, Z_X_gen_greater_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_greater_X_D_1[t in nb_hours_ls], sum(X_gen[t,i] for i in 1:2) - X_D[t] <= M*Z_X_gen_greater_X_D[t])
@constraint(model, Z_X_gen_greater_X_D_2[t in nb_hours_ls], -sum(X_gen[t,i] for i in 1:2) + X_D[t] <= M*(1-Z_X_gen_greater_X_D[t]))

#Expression of X_ch
@constraint(model, X_ch_expression[t in nb_hours_ls], X_ch[t]  == Z_X_gen_greater_X_D[t]*min_X_1_X_2_X_3[t]);

Expression of min of [X_gen[t]- X_D[t],  P[5], X_soc[t]*P3]* H]

In [19]:
# X_1, X_2, X_3 = [X_gen[t]- X_D[t]) , P3], X_soc[t])*P3]* H]
@variable(model, y1[nb_hours_ls], Bin)
@variable(model, y2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Y_1 expression")
@constraint(model, y_1_1[t in nb_hours_ls], P[3] - (X_D[t] - sum(X_gen[t,i] for i in 1:2))   <=  M*z1[t])
@constraint(model, y_1_2[t in nb_hours_ls], -P[3] + (X_D[t] - sum(X_gen[t,i] for i in 1:2))  <= M*(1-y1[t]))



#Minimum between X_1 and X_2
@variable(model, min_extra_prod_P_5[nb_hours_ls])

println("Min X_1, X_2 expression")
@constraint(model, min_extra_prod_P_5_lower_1[t in nb_hours_ls], min_extra_prod_P_5[t] <= sum(X_gen[t,i] for i in 1:2)- X_D[t])
@constraint(model, min_extra_prod_P_5_lower_2[t in nb_hours_ls], min_extra_prod_P_5[t] <= P[3])
@constraint(model, min_extra_prod_P_5_higher_1[t in nb_hours_ls], min_extra_prod_P_5[t] >= (X_D[t] - sum(X_gen[t,i] for i in 1:2)) - M*(1-y1[t]))
@constraint(model, min_extra_prod_P_5_higher_2[t in nb_hours_ls], min_extra_prod_P_5[t] >= P[3] - M*y1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, y_2_1[t in nb_hours_ls], X_soc[t]* P[3] * H - min_extra_prod_P_5[t] <=  M*y2[t])
@constraint(model, y_2_2[t in nb_hours_ls], - X_soc[t] * P[3] * H + min_extra_prod_P_5[t] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_extra_prod_P5_soc[nb_hours_ls])

@constraint(model, min_extra_prod_P5_soc_lower_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= min_extra_prod_P_5[t])
@constraint(model, min_extra_prod_P5_soc_lower_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= X_soc[t]*P[3]* H)
@constraint(model, min_extra_prod_P5_soc_higher_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= min_extra_prod_P_5[t] - M*(1-y2[t]))
@constraint(model, min_extra_prod_P5_soc_higher_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= X_soc[t]*P[3]* H - M*y2[t]);



Y_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


In [20]:
# X_1, X_2, X_3 = [X_gen[t]- X_D[t]) , P3], X_soc[t])*P3]* H]
#@variable(model, y1[nb_hours_ls], Bin)
#@variable(model, y2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Y_1 expression")
#@constraint(model, y_1_1[t in nb_hours_ls], P[3] - (sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*y1[t])
#@constraint(model, y_1_2[t in nb_hours_ls], -P[3] + (sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-y1[t]))


#Minimum between X_1 and X_2
#@variable(model, min_extra_prod_P_5[nb_hours_ls])

println("Min X_1, X_2 expression")
#@constraint(model, min_extra_prod_P_5_lower_1[t in nb_hours_ls], min_extra_prod_P_5[t] <= sum(X_gen[t,i] for i in 1:2)- X_D[t])
#@constraint(model, min_extra_prod_P_5_lower_2[t in nb_hours_ls], min_extra_prod_P_5[t] <= P[3])
#@constraint(model, min_extra_prod_P_5_higher_1[t in nb_hours_ls], min_extra_prod_P_5[t] >= (sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-y1[t]))
#@constraint(model, min_extra_prod_P_5_higher_2[t in nb_hours_ls], min_extra_prod_P_5[t] >= P[3] - M*y1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
#@constraint(model, y_2_1[t in nb_hours_ls], X_soc[t]* P[3] * H - P[3] <=  M*y2[t])
#@constraint(model, y_2_2[t in nb_hours_ls], - X_soc[t] * P[3] * H + P[3] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
#@variable(model, min_extra_prod_P5_soc[nb_hours_ls])

#@constraint(model, min_extra_prod_P5_soc_lower_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= P[3])
#@constraint(model, min_extra_prod_P5_soc_lower_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= X_soc[t]*P[3]* H)
#@constraint(model, min_extra_prod_P5_soc_higher_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= P[3] - M*(1-y2[t]))
#@constraint(model, min_extra_prod_P5_soc_higher_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= X_soc[t]*P[3]* H - M*y2[t]);



Y_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_dch

In [21]:

#Expression of X_dch
# issue with this constraint
#@constraint(model, X_dch_expression[t in nb_hours_ls], X_gen[t,3]  == Z_X_gen_lower_X_D[t]*min_extra_prod_P5_soc[t]);
@constraint(model, X_dch_expression[t in nb_hours_ls], X_gen[t,3]  == Z_X_gen_lower_X_D[t]);

Expression of state of charge

In [22]:


#@constraint(model, SOC_t_1, X_soc[1] == 1)

#@constraint(model, SOC_t[t in nb_hours_ls[2:end]], (X_soc[t]-X_soc[t-1])*H*P[3] == (X_ch[t-1]- X_dch[t-1]));

@constraint(model, SOC_t[t in nb_hours_ls], X_soc[t]== 0.5);

In [23]:
optimize!(model);

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 27277 rows, 15738 columns and 77431 nonzeros
Model fingerprint: 0x9d19ae53
Model has 9441 quadratic constraints
Variable types: 9444 continuous, 6294 integer (6294 binary)
Coefficient statistics:
  Matrix range     [2e-09, 1e+13]
  QMatrix range    [1e+00, 2e+01]
  QLMatrix range   [1e+00, 1e+13]
  Objective range  [4e+02, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+13]
  QRHS range       [1e+13, 1e+13]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve added 1461 rows and 0 columns
Presolve removed 0 rows and 3264 columns
Presolve time: 0.50s
Presolved: 31970 rows, 15451 columns, 102123 nonzeros
Presolved model has 1928 SOS constraint(s)
Variable types: 8677 continuous, 6774 integ

In [24]:
objective_value(model)

1.8791566162814185e7

In [25]:
println(value.(P))
#println(value.(s1[:]))
#println(value.(X_soc))



1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
     0.0
 26390.180143232606
  1000.001137694735


In [26]:
#println(value.(X_gen))
X_gen_matrix = value.(X_gen);




In [27]:
using Plots


@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[:,1]
b = X_gen_matrix[:,2]
c = X_gen_matrix[:,3]
sNames = ["Solar","Wind","Battery"]
x = nb_hours_ls

plotly()
stackedarea(x, [a b c], labels=reshape(sNames, (1,3)))
plot!(x, X_D, label="Demand", lc=:black, lw=2, ls=:dot)
#genplot(x, X_D[:])

In [32]:
println(size(min_X_1_X_2_X_3))
Z_X_gen_lower_X_D_values = value.(Z_X_gen_lower_X_D[:])
X_minX3 = value.(min_X_1_X_2_X_3)
X_2 = value.(eta*(sum(X_gen[:,i] for i in 1:2)- X_D[:]))
X_1 = value.(eta*P[3]*(ones(size(X_soc))))
X_3 = value.((ones(size(X_soc))- X_soc[:])*P[3]* H );
println("X ch")
X_charge = value.(X_ch)
z_1_true = value.(z1)
ls_X_min = []

for i = X_minX3
    ls_X_min = [ls_X_min; i]
    #println(i)
end


ls_X_ch = []

for i = X_charge
    ls_X_ch = [ls_X_ch; i]
    #println(i)
end



println(ls_X_min)
plotly()
x = nb_hours_ls
plot(x, [ X_1 X_2 z_1_true], label=["X1" "X2" "Z_1"])
plot!(x, ls_X_ch, label="X charge", lc=:black, lw=2, ls=:dot)


(1049,)
X ch
Any[800.0001445737898, 799.9987401734306, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.0004679810547, 800.000910155788, 0.0, -0.8003120554359384, 800.000910155788, 800.000910155788, 0.0, -0.00039368099453440664, 800.000910155788, 800.000910155788, 800.000910155788, -0.0003797199708515109, 800.000910155788, -0.7999999999999545, -0.7999999999999545, 800.000910155788, 800.000910155788, 800.000910155788, -0.0007155679853667607, 800.000910155788, 800.000910155788, -0.0005612239941683583, 800.000910155788, 800.000910155788, -0.7999998907228019, 800.0002888977863, 800.000910155788, 800.000910155788, 0.0007599999999001739, 800.000910155788, 0.0, -0.8004435300000523, 0.0, -0.00029696000910917064, 0.0, 0.0, 0.0003906250145746526, 0.0, 800.000910155788, 0.0, 800.000910155788, 0.0, 0.0, 800.000910155788, 800.000910155788, 800.0009

In [29]:
println(size(min_X_1_X_2_X_3))
Z_X_gen_lower_X_D_values = value.(Z_X_gen_lower_X_D[:])
y_true_1 = value.(y1)
Y_minY3 = value.(min_extra_prod_P_5)
Z_2 = value.(P[3]*ones(size(X_soc)))
Z_1 = value.(sum(X_gen[:,i] for i in 1:2)- X_D[:])
Z_3 = value.(X_soc[:]* P[3] * H );
println("X ch")
X_decharge = value.(X_gen[:,3])

ls_Y_min = []

for i = Y_minY3
    ls_Y_min = [ls_Y_min; i]
    #println(i)
end


ls_X_dch = []

for i = X_decharge
    ls_X_dch = [ls_X_dch; i]
    #println(i)
end



println(ls_X_min)
plotly()
x = nb_hours_ls
plot(x, [ Z_1 Z_2 y_true_1], label=[ "X1" "X2" "y_1"])
#plot!(x, ls_Y_min, label="X min", lc=:red, lw=2, ls=:dot)
#plot!(x, ls_X_dch, label="X decharge", lc=:black, lw=2, ls=:dot)


(1049,)
X ch
Any[800.0001445737898, 799.9987401734306, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.000910155788, 800.0004679810547, 800.000910155788, 0.0, -0.8003120554359384, 800.000910155788, 800.000910155788, 0.0, -0.00039368099453440664, 800.000910155788, 800.000910155788, 800.000910155788, -0.0003797199708515109, 800.000910155788, -0.7999999999999545, -0.7999999999999545, 800.000910155788, 800.000910155788, 800.000910155788, -0.0007155679853667607, 800.000910155788, 800.000910155788, -0.0005612239941683583, 800.000910155788, 800.000910155788, -0.7999998907228019, 800.0002888977863, 800.000910155788, 800.000910155788, 0.0007599999999001739, 800.000910155788, 0.0, -0.8004435300000523, 0.0, -0.00029696000910917064, 0.0, 0.0, 0.0003906250145746526, 0.0, 800.000910155788, 0.0, 800.000910155788, 0.0, 0.0, 800.000910155788, 800.000910155788, 800.0009

In [30]:
P_3_matrix = value(P[3])*ones(size(nb_hours_ls))

println(P_3_matrix)
println("X gen")
X_gen_non_battery = sum(X_gen_matrix[:,i] for i =1:2) - X_D[:]
println("X soc")
X_soc_charge = (ones(size(X_soc))- value.(X_soc))* value(P[3]) * H

println("X ch")
X_charge = value.(X_ch)

plot(x, [P_3_matrix X_gen_non_battery X_soc_charge X_charge], label=["P[3]" "X_gen Non batt" "X possible charge" "X_ch"])


[1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137694735, 1000.001137

In [31]:
println(value.(min_extra_prod_P5_soc))
println(value.(Z_X_gen_lower_X_D))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1049
And data, a 1049-element Vector{Float64}:
 1000.001137694735
 1000.001137694735
 1000.0011376947168
 1000.001137694725
   -0.9971974000845876
 1000.001137694725
 1000.0011376947241
   -0.9908221500909349
 1000.0011376947286
 1000.0011376947314
   -0.9908615000358623
 1000.0011376947277
   -0.9947036200683215
 -702.69476106
 1000.001137694735
    0.0006965849909192912
   -0.9999999568018438
 1000.0011376947341
 -852.1171875
    0.00010248998864881287
   -3.820000074483687e-6
 1000.0011376947332
 1000.0011376947332
 1000.001137694735
   -0.000474649963621232
 1000.001137694735
   -1.000560549999932
   -1.0009540749999815
 1000.001137694735
 1000.0011376947332
 1000.0011376947332
   -0.0008944599817368726
 1000.0011376947332
 -544.0427434000003
   -0.0007015299927388696
 1000.0011376947332
 1000.0011376947332
   -0.9999998634035023
 1000.001137694735
 1000.0011376947286
 1000.0011376947332
    0.0
 1000.0


 1000.001137694735
   -5.093170329928398e-11
   -0.0005683250219590263
 1000.001137694735
 1000.0011376947341
 1000.0011376947332
   -0.990233425286533
 1000.0011376947341
 1000.0011376947332
   -1.000000000003638
 1000.0011376947323
    0.0009265112498724193
   -1.0
 1000.0011376947332
 1000.0011376947341
 1000.0009621247355
 1000.001137694735
 1000.0011376947286
 1000.0011376947314
 1000.001137694735
 1000.001137694735
    0.0
   -1.0003682000000254
 1000.001137694735
 1000.001137694735
 1000.001137694735
 1000.001137694735
   -0.0005714999817882926
 1000.0011376947341
 -890.58984375
 1000.0011376947341
    0.0
 -693.99609375
   -0.9999999372148523
   -0.0007811550036649351
 1000.0011376947332
 -524.857421875
 1000.0011376947332
 1000.0011376947323
 1000.0011376947277
 1000.0011376947305
   -0.9918523500855372
    0.0
    0.0
 1000.001137694735
 1000.001137694735
 -922.1677634999996
 1000.001137694735
 -965.8104337000004
 1000.001137694735
 1000.001137694735
 1000.001137694735
   -0